In [4]:
import pandas as pd
data = "../Resources/Historic_Incidents_2016_2017.csv"

In [5]:
data_df=pd.read_csv(data)
data_df.head()

,X,Y,FID,ID,WST_Start,WST_End,UTC_Start,UTC_END,Incident_Type,Description,Road,Operations_Centre,Location,Local_Rd_Name,TrafficCondition,Lat,Long,GlobalID
0,115.894297,-32.065714,1,62397,2017/01/01 16:59:00,2017/01/01 21:35:00,2017/01/01 08:58:59,2017/01/01 18:35:00,Road Crash,Roe Hwy southbound South Street Canningvale -...,Roe Hwy,Metro,All Lanes Open. No Known Congestion. Inciden...,NaN,All Lanes Open,-32.065714,115.894297,{EC2D031A-9C1F-430C-95F6-39F4E01DCF60}
1,115.786788,-33.991912,2,62401,2017/01/01 18:24:59,2017/01/12 11:12:00,2017/01/01 10:25:00,2017/01/12 08:12:00,Bushfire,Brockman Hwy both directions Brockman Rd East ...,Brockman Hwy,South West,All Lanes Open. No Known Congestion. Inciden...,NaN,All Lanes Open,-33.991912,115.786788,{7A443C5B-0C60-4A36-BBC9-0CA666A909F6}
2,115.711201,-31.674911,3,62408,2017/01/02 04:50:59,2017/01/02 05:48:00,2017/01/01 20:51:00,2017/01/02 02:48:00,Road Crash,Wanneroo Rd northbound near Hester Av Neerabu...,Wanneroo Rd,Metro,All Lanes Open. No Known Congestion. Inciden...,NaN,All Lanes Open,-31.674911,115.711201,{814E5E47-AF04-45FC-8372-0A47A2C82E60}
3,115.838520,-31.936552,4,62412,2017/01/02 07:30:00,2017/01/02 08:29:59,2017/01/01 23:29:59,2017/01/02 05:29:59,Road Crash,Mitchell Fwy southbound Vincent St Leederville...,Mitchell Fwy,Metro,All Lanes Open. No Known Congestion. Inciden...,NaN,All Lanes Open,-31.936552,115.838520,{E38AA494-D9A9-4901-9286-EE78AC27E5C4}
4,116.015323,-31.892157,5,62418,2017/01/02 08:16:59,2017/01/02 08:55:00,2017/01/02 00:17:00,2017/01/02 05:55:00,Road Crash,Great Eastern Hwy direction unknown Loton Av M...,Great Eastern Hwy,Metro,All Lanes Open. No Known Congestion. No Deto...,NaN,All Lanes Open,-31.892157,116.015323,{0E716842-4B22-4EE7-AD9F-C3D7D2E0C6CC}


In [9]:
data_df = data_df.loc[data_df["Operations_Centre"]=="Metro"]
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9897 entries, 0 to 11393
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  9897 non-null   float64
 1   Y                  9897 non-null   float64
 2   FID                9897 non-null   int64  
 3   ID                 9897 non-null   int64  
 4   WST_Start          9897 non-null   object 
 5   WST_End            9897 non-null   object 
 6   UTC_Start          9897 non-null   object 
 7   UTC_END            9897 non-null   object 
 8   Incident_Type      9897 non-null   object 
 9   Description        9897 non-null   object 
 10  Road               7004 non-null   object 
 11  Operations_Centre  9897 non-null   object 
 12  Location           9897 non-null   object 
 13  Local_Rd_Name      348 non-null    object 
 14  TrafficCondition   7004 non-null   object 
 15  Lat                9897 non-null   float64
 16  Long               9897

In [24]:
new_data = data_df[["WST_Start","Incident_Type","TrafficCondition","Lat","Long"]]
new_data.head()

,WST_Start,Incident_Type,TrafficCondition,Lat,Long
0,2017/01/01 16:59:00,Road Crash,All Lanes Open,-32.065714,115.894297
2,2017/01/02 04:50:59,Road Crash,All Lanes Open,-31.674911,115.711201
3,2017/01/02 07:30:00,Road Crash,All Lanes Open,-31.936552,115.838520
4,2017/01/02 08:16:59,Road Crash,All Lanes Open,-31.892157,116.015323
5,2017/01/02 12:13:00,Debris / Trees / Lost Loads,Right Centre Lane(s) Blocked,-31.936639,115.838076
